In [7]:
#Neeraj Nair
#CB.EN.U4CSE22140

In [4]:
def step(x):
    return 1 if x >= 0 else 0

def perceptronAnd(x1, x2):
    w1, w2, bias = 1, 1, -1.5
    y = w1 * x1 + w2 * x2 + bias
    return step(y)

print("AND Gate:")
for x1 in [0, 1]:
    for x2 in [0, 1]:
        print(f"{x1} AND {x2} = {perceptronAnd(x1, x2)}")


AND Gate:
0 AND 0 = 0
0 AND 1 = 0
1 AND 0 = 0
1 AND 1 = 1


In [5]:
def step(x):
    return 1 if x >= 0 else 0

def perceptronOr(x1, x2):
    w1, w2, bias = 1, 1, -0.5
    y = w1 * x1 + w2 * x2 + bias
    return step(y)

print("\nOR Gate:")
for x1 in [0, 1]:
    for x2 in [0, 1]:
        print(f"{x1} OR {x2} = {perceptronOr(x1, x2)}")



OR Gate:
0 OR 0 = 0
0 OR 1 = 1
1 OR 0 = 1
1 OR 1 = 1


In [6]:
def step(x):
    return 1 if x >= 0 else 0

def hideOr(x1, x2):
    w1, w2, bias = 1, 1, -0.5
    y = w1 * x1 + w2 * x2 + bias
    return step(y)

def hideNand(x1, x2):
    w1, w2, bias = -2, -2, 3
    y = w1 * x1 + w2 * x2 + bias
    return step(y)

def perceptronXor(x1, x2):
    h1 = hideOr(x1, x2)
    h2 = hideNand(x1, x2)
    w1, w2, bias = 1, 1, -1.5
    y = w1 * h1 + w2 * h2 + bias
    return step(y)

print("\nXOR Gate:")
for x1 in [0, 1]:
    for x2 in [0, 1]:
        print(f"{x1} XOR {x2} = {perceptronXor(x1, x2)}")



XOR Gate:
0 XOR 0 = 0
0 XOR 1 = 1
1 XOR 0 = 1
1 XOR 1 = 0
